# Who are the high-frequency traders ?

## Process the data

In [3]:
import pandas as pd

x_train = pd.read_csv("data/AMF_train_X_XCZw8r3.csv", index_col=0, sep=',')
y_train = pd.read_csv("data/AMF_train_Y_omYQJZL.csv", index_col=0, sep=',')

print(x_train[:5])

         Share      Day  ... med_dt_TV1_TV4  NbSecondWithAtLeatOneTrade
Index                    ...                                           
1       Isin_8  Date_24  ...            NaN                           4
2      Isin_87  Date_29  ...            NaN                          15
3      Isin_87  Date_23  ...            NaN                          63
4      Isin_12  Date_11  ...            NaN                           4
5      Isin_87   Date_9  ...            NaN                          38

[5 rows x 38 columns]
